# Pokémon TCG Synergy Engine
---

In [1]:
# import libraries for data wrangling
import ast
import re
import pandas as pd
import numpy as np
import src

## Pokémon TCG Data Wrangling
---

In [2]:
data = src.get_card_data()
df_pokemon = src.prep_pokemon_data(data)
df_trainers = src.prep_trainer_df(data)